In [29]:
import pandas as pd
import numpy as np

In [30]:
# Load the dataset
data = pd.read_csv('../HIGGS_train.csv')

#label the data columns with the following names: label, lepton pT, lepton eta, lepton phi, missing energy magnitude, missing energy phi, jet 1 pt, jet 1 eta, jet 1 phi, jet 1 b-tag, jet 2 pt, jet 2 eta, jet 2 phi, jet 2 b-tag, jet 3 pt, jet 3 eta, jet 3 phi, jet 3 b-tag, jet 4 pt, jet 4 eta, jet 4 phi, jet 4 b-tag, m_jj, m_jjj, m_lv, m_jlv, m_bb, m_wbb, m_wwbb
data.columns = ['label', 'lepton pT', 'lepton eta', 'lepton phi', 'missing energy magnitude', 'missing energy phi', 'jet 1 pt', 'jet 1 eta', 'jet 1 phi', 'jet 1 b-tag', 'jet 2 pt', 'jet 2 eta', 'jet 2 phi', 'jet 2 b-tag', 'jet 3 pt', 'jet 3 eta', 'jet 3 phi', 'jet 3 b-tag', 'jet 4 pt', 'jet 4 eta', 'jet 4 phi', 'jet 4 b-tag', 'm_jj', 'm_jjj', 'm_lv', 'm_jlv', 'm_bb', 'm_wbb', 'm_wwbb']

# Print the first five rows of the data
print(data.head())

   label  lepton pT  lepton eta  lepton phi  missing energy magnitude  \
0    1.0      0.908       0.329     0.35900                     1.500   
1    1.0      0.799       1.470    -1.64000                     0.454   
2    0.0      1.340      -0.877     0.93600                     1.990   
3    1.0      1.110       0.321     1.52000                     0.883   
4    0.0      1.600      -0.608     0.00707                     1.820   

   missing energy phi  jet 1 pt  jet 1 eta jet 1 phi  jet 1 b-tag  ...  \
0              -0.313     1.100     -0.558     -1.59         2.17  ...   
1               0.426     1.100      1.280      1.38         0.00  ...   
2               0.882     1.790     -1.650    -0.942         0.00  ...   
3              -1.210     0.681     -1.070    -0.922         0.00  ...   
4              -0.112     0.848     -0.566      1.58         2.17  ...   

   jet 4 eta  jet 4 phi  jet 4 b-tag   m_jj  m_jjj   m_lv  m_jlv   m_bb  \
0     -1.140  -0.000819          0.0  0.3

/var/folders/cl/vqr669317693_lc6y70yn19r0000gn/T/ipykernel_89050/3790264241.py:2: DtypeWarning: Columns (8,21) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../HIGGS_train.csv')


In [31]:
# Find columns with non-float data types
non_float_columns = data.select_dtypes(exclude=['float']).columns.tolist()

# Convert non-float columns to numeric data type
for col in non_float_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')

# Find rows with non-float values
nonfloat_rows = data[data.isnull().any(axis=1)]

# Print the row numbers and column names with non-float values
for index, row in nonfloat_rows.iterrows():
    nonfloat_columns = row.index[row.isnull()].tolist()
    print(f"Row {index} contains non-float values in columns: {nonfloat_columns}")

Row 260691 contains non-float values in columns: ['jet 3 b-tag']
Row 261025 contains non-float values in columns: ['jet 1 phi']
Row 490958 contains non-float values in columns: ['jet 4 b-tag']
Row 490959 contains non-float values in columns: ['jet 4 b-tag']


In [32]:
#scale the data using the min-max scaler
from sklearn.preprocessing import MinMaxScaler

# Initialize scaler
scaler = MinMaxScaler()

# Scale data
scaled_data = scaler.fit_transform(data)

# Print first five rows of scaled data
print(scaled_data[:5])

data = pd.DataFrame(scaled_data, columns=data.columns)

[[1.         0.07504446 0.56769547 0.60316092 0.1514615  0.41005747
  0.11661206 0.40606061 0.04310345 1.         0.05468408 0.46323024
  0.86494253 1.         0.02861994 0.26923077 0.71034483 0.
  0.00302626 0.272      0.49976466 0.         0.00878655 0.05178336
  0.15336834 0.07937736 0.05361159 0.08477913 0.07564732]
 [1.         0.06212211 0.80246914 0.02873563 0.04579825 0.62241379
  0.11661206 0.71548822 0.89655172 0.         0.05810183 0.76460481
  0.26436782 1.         0.0884065  0.56520147 0.43994253 1.
  0.07966177 0.726      0.75862069 0.         0.03618258 0.0761779
  0.15336834 0.0767669  0.05529633 0.06927526 0.07260112]
 [0.         0.12625963 0.31954733 0.76896552 0.20095958 0.75344828
  0.20033976 0.22222222 0.22931034 0.         0.1955131  0.3838488
  0.71149425 1.         0.12563667 0.23809524 0.39511494 0.
  0.03382287 0.3644     0.1091954  0.         0.03784977 0.06913254
  0.15559903 0.05520642 0.06013082 0.0894549  0.10272466]
 [1.         0.09899229 0.56604938 0

In [33]:
print(data.describe())

               label      lepton pT     lepton eta     lepton phi  \
count  599999.000000  599999.000000  599999.000000  599999.000000   
mean        0.529286       0.085061       0.499977       0.500048   
std         0.499142       0.066988       0.207378       0.288931   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.000000       0.037463       0.348354       0.250000   
50%         1.000000       0.068643       0.499788       0.500759   
75%         1.000000       0.114404       0.651852       0.750000   
max         1.000000       1.000000       1.000000       1.000000   

       missing energy magnitude  missing energy phi       jet 1 pt  \
count             599999.000000       599999.000000  599999.000000   
mean                   0.100753            0.499668       0.103282   
std                    0.060537            0.289297       0.057593   
min                    0.000000            0.000000       0.000000   
25%                    0.058

In [34]:
# Delete rows with non-float values
data.drop(nonfloat_rows.index, inplace=True)

In [6]:
# Calculate z-scores for each column
z_scores = np.abs((data - data.mean()) / data.std())

# Define threshold for z-scores
z_threshold = 3

# Remove rows with z-score greater than threshold in any column
outlier_rows = data[(z_scores > z_threshold).any(axis=1)]
data = data[(z_scores <= z_threshold).all(axis=1)]

# Print number of outlier rows removed
print(f"{len(outlier_rows)} outlier rows removed.")

79658 outlier rows removed.


In [ ]:
#write a code that finds the correlated columns and removes them
# Find columns with correlation greater than threshold
correlation_threshold = 0.8
correlated_columns = set()
correlation_matrix = data.corr()
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > correlation_threshold:
            colname = correlation_matrix.columns[i]
            correlated_columns.add(colname)
        
# Drop correlated columns
data.drop(labels=correlated_columns, axis=1, inplace=True)

# Print number of correlated columns removed
print(f"{len(correlated_columns)} correlated columns removed.")

In [35]:
#  data['m_bb^2']=data['m_bb']**2
# Add new features
data['Delta phi jet 1 jet 2'] = abs(data['jet 1 phi'] - data['jet 2 phi'])
data['Delta eta jet 1 jet 2'] = abs(data['jet 1 eta'] - data['jet 2 eta'])

print(data.head())

   label  lepton pT  lepton eta  lepton phi  missing energy magnitude  \
0    1.0   0.075044    0.567695    0.603161                  0.151461   
1    1.0   0.062122    0.802469    0.028736                  0.045798   
2    0.0   0.126260    0.319547    0.768966                  0.200960   
3    1.0   0.098992    0.566049    0.936782                  0.089134   
4    0.0   0.157084    0.374897    0.502032                  0.183787   

   missing energy phi  jet 1 pt  jet 1 eta  jet 1 phi  jet 1 b-tag  ...  \
0            0.410057  0.116612   0.406061   0.043103          1.0  ...   
1            0.622414  0.116612   0.715488   0.896552          0.0  ...   
2            0.753448  0.200340   0.222222   0.229310          0.0  ...   
3            0.152299  0.065769   0.319865   0.235057          0.0  ...   
4            0.467816  0.086033   0.404714   0.954023          1.0  ...   

   jet 4 b-tag      m_jj     m_jjj      m_lv     m_jlv      m_bb     m_wbb  \
0          0.0  0.008787  0.0517

In [36]:
# Generate synthetic data
synthetic_data = pd.DataFrame(columns=data.columns)
for col in data.columns:
    synthetic_data[col] = np.random.normal(data[col].mean(), data[col].std(), len(data))

# Print the first five rows of the synthetic data
print(synthetic_data.head())

      label  lepton pT  lepton eta  lepton phi  missing energy magnitude  \
0  0.352032   0.066680    0.980579    0.539973                  0.074649   
1  0.178451  -0.008291    0.756576    0.694435                  0.015624   
2  0.742430   0.128053    0.839088    0.269355                  0.098904   
3  0.811036   0.141719    0.272623    0.269545                  0.187036   
4 -0.083634   0.069182    0.400144    0.635856                  0.054877   

   missing energy phi  jet 1 pt  jet 1 eta  jet 1 phi  jet 1 b-tag  ...  \
0            0.298599  0.131689   0.452315   0.411172     0.698258  ...   
1            0.545230  0.121336   0.573704   0.809893     0.354138  ...   
2            1.061601  0.107931   0.604443   0.550602     0.275448  ...   
3            0.639859  0.247526   0.311638   0.524287     0.377738  ...   
4            0.698716  0.080684   0.486719   0.359486     0.297999  ...   

   jet 4 b-tag      m_jj     m_jjj      m_lv     m_jlv      m_bb     m_wbb  \
0    -0.694628

In [37]:
# Import necessary libraries
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split the data into features and target variable
X = data.iloc[:, 1:]
y = data.iloc[:, 0]

#shuffle the data
from sklearn.utils import shuffle
X, y = shuffle(X, y, random_state=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



# Create the logistic regression model
logistic_regression_model = LogisticRegression()

# Fit the model on the training data
logistic_regression_model.fit(X_train, y_train)

# Make predictions on the testing data
predictions = logistic_regression_model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)


Accuracy: 0.64065355918644


/opt/homebrew/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from sklearn.model_selection import KFold

# apply k-fold cross validation
k_fold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = []

for train_indices, test_indices in k_fold.split(X):
    # split data into training and testing sets
    X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]
    
    # create and train the model
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)
    
    # evaluate the model on the testing set
    score = model.score(X_test, y_test)
    scores.append(score)

# print the accuracy of each fold
print(scores)

# print the average accuracy of all 10 folds
print(np.mean(scores))


In [ ]:
#perform regularized logistic regression on different values of C
from sklearn.linear_model import LogisticRegression

# create and train the model
model = LogisticRegression(C=1)
model.fit(X_train, y_train)

# print the accuracy for the model
print(model.score(X_test, y_test))

In [ ]:
#perform xgboost on the dataset
import xgboost as xgb

# create the DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train)

# create the parameter dictionary
params = {'max_depth': 2, 'eta': 1, 'objective': 'binary:logistic'}

# train the model
num_rounds = 2
bst = xgb.train(params, dtrain, num_rounds)

# create the DMatrix for the testing data
dtest = xgb.DMatrix(X_test)

# make predictions
predictions = bst.predict(dtest)

# print the first 4 predictions
print(predictions[:4])

# print the first 4 actual labels
print(y_test[:4])

# print the accuracy
accuracy = accuracy_score(y_test, predictions.round())
print("Accuracy:", accuracy)

In [11]:
#implementing a decision tree
from sklearn.tree import DecisionTreeClassifier

# create the model
model = DecisionTreeClassifier()

# train the model
model.fit(X_train, y_train)

# make predictions
predictions = model.predict(X_test)

# print the accuracy
accuracy = accuracy_score(y_test, predictions)
print("Tree accuracy:", accuracy)

#implementing a random forest
from sklearn.ensemble import RandomForestClassifier

# create the model
model = RandomForestClassifier()

# train the model
model.fit(X_train, y_train)

# make predictions
predictions = model.predict(X_test)

# print the accuracy
accuracy = accuracy_score(y_test, predictions)
print("Forest accuracy:", accuracy)

Tree accuracy: 0.6351936554304237
Forest accuracy: 0.724391743859784


In [ ]:
#implementing a gradient boosted tree
from sklearn.ensemble import GradientBoostingClassifier

# create the model
model = GradientBoostingClassifier()

# train the model
model.fit(X_train, y_train)

# make predictions
predictions = model.predict(X_test)

# print the accuracy
accuracy = accuracy_score(y_test, predictions)
print("Gradient Boosted Tree accuracy:", accuracy)

In [38]:
# Implementing a neural network
from sklearn.neural_network import MLPClassifier


activations = ['identity', 'logistic', 'tanh', 'relu']
solvers = ['lbfgs', 'sgd', 'adam']

# create the model
model = MLPClassifier(hidden_layer_sizes=(40, 35, 30, 25), max_iter=500)

model.fit(X_train, y_train)
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print("Training set accuracy:", model.score(X_train, y_train))
print(f"Testing accuracy:", accuracy)

#save the model using pickle
import pickle

s = pickle.dumps(model)
saved_model = pickle.loads(s)

best_weights=model.coefs_
best_biases=model.intercepts_

Training set accuracy: 0.7514690616101106
Testing accuracy: 0.7454819193439963


In [ ]:
from tensorflow.keras.layers import Dense, BatchNormalization 
from keras import Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras import regularizers  # Added import for regularizers

'''Write your code here'''

import tensorflow as tf

# Define the model architecture
model = Sequential([
    Dense(200, activation='relu'),  # Increased neurons to 128
    BatchNormalization(),
    Dense(100, activation='relu'),  # Increased neurons to 64
    BatchNormalization(),
    Dense(28, activation='relu'),  # Increased neurons to 32
    BatchNormalization(),
    Dense(1, activation='sigmoid')
])


# Compile the model with appropriate loss function and optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model on the training set
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)

# Print the test accuracy
print('Test accuracy:', test_acc)